In [ ]:
# for domonkos to run on COLAB

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/python/Computer_Vision/TDT4265_Project/')

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.data import load_train_test_val, load_tee, load_tee_no_gt
from src.visualize import plot_loss_acc
from src.model import Unet2D
#from src.baseline_model import Unet2D
from src.train import train
from src.utils import save_result, to_cuda
from src.test import test, predict
from src.metrics import acc_metric

from datetime import datetime

from params import *

import torch
from torch import nn

## Prepare data

In [4]:
train_data, test_data, valid_data = load_train_test_val(DATA_PARAMS, PREP_STEPS, TRAIN_TRANSFORMS)

DATASET: extracted_CAMUS

Items loaded: 1800[train: 1440, valid: 160 test: 200]
RAW IMAGES: torch.Size([8, 1, 384, 384])
 GT IMAGES: torch.Size([8, 384, 384])



## Train

In [7]:
# MODEL: Unet2D (one input channel, 4 output channels)
unet = Unet2D(1, out_channels=4)
opt = torch.optim.Adam(unet.parameters(), lr=LEARN_RATE)

In [8]:
if LOAD: # Load pretrained model
    unet.load_state_dict(torch.load(MODEL_PATH+FILE_NAME))
    to_cuda(unet)
    print(f"The pre-trained model'{FILE_NAME}' has been loaded")
else:  # Start the training process
    start = datetime.now()
    train_loss, valid_loss, train_acc, valid_acc = train(unet, train_data, valid_data, LOSS, opt, acc_metric, epochs=EPOCHS)

    end = datetime.now()
    print(f"Elapsed time is {str(end-start)}")

Epoch 1/1
----------
Current step: 40  Loss: 1.4361  Acc: 0.804 Dice: (tensor(0.5081), tensor([0.8972, 0.6246, 0.5105, 0.0000])) Val Loss: 2.3688 Val Acc: 0.208 Val Dice: (tensor(0.1297), tensor([0.2941, 0.1543, 0.0702, 0.0000]))
Current step: 80  Loss: 1.3556  Acc: 0.833 Dice: (tensor(0.5560), tensor([0.9111, 0.7372, 0.5758, 0.0000])) Val Loss: 1.3779 Val Acc: 0.842 Val Dice: (tensor(0.5280), tensor([0.9322, 0.6835, 0.4964, 0.0000]))
Current step: 120  Loss: 1.2536  Acc: 0.874 Dice: (tensor(0.5930), tensor([0.9403, 0.7719, 0.6598, 0.0000])) Val Loss: 1.3831 Val Acc: 0.832 Val Dice: (tensor(0.4994), tensor([0.9050, 0.6772, 0.4156, 0.0000]))
Current step: 160  Loss: 1.1218  Acc: 0.889 Dice: (tensor(0.7355), tensor([0.9499, 0.8035, 0.6314, 0.5571])) Val Loss: 1.1600 Val Acc: 0.870 Val Dice: (tensor(0.7063), tensor([0.9397, 0.8059, 0.6020, 0.4776]))
Epoch 1/1
----------
Train Loss: 1.3632 Acc: 0.8259991407394409
----------
Elapsed time is 0:02:29.621752


In [ ]:
if not LOAD:
    plot_loss_acc(train_loss, valid_loss, train_acc, valid_acc)

## Test

In [9]:
print("TEST on the CAMUS dataset")
accuracy, average_dice, class_dice = test(unet, test_data, False)

TEST on the CAMUS dataset

Final Accuracy: 0.871 (delta to baseline (-0.09958536149321284, 4))
Final average DICE score: 0.6951000094413757 [0.9404240250587463, 0.7484806180000305, 0.6098035573959351, 0.48175808787345886] (delta to baseline 0.0877)


In [10]:
print(f"Test on TEE images")
tee_data = load_tee(DATA_PARAMS['base_path'], DATA_PARAMS['batch_size'], PREP_STEPS)
accuracy_tee, average_dice_tee, class_dice_tee = test(unet, tee_data, False)

Test on TEE images
TEE IMAGES: 19
RAW TEE IMAGES: torch.Size([8, 1, 384, 384])
 GT IMAGES: torch.Size([8, 384, 384])


Final Accuracy: 0.7704 (delta to baseline (-0.20021837951002808, 4))
Final average DICE score: 0.32429999113082886 [0.8923982977867126, 0.27925363183021545, 0.12541373074054718, 0.0] (delta to baseline -0.2832)


In [20]:
tee_data_no_gt = load_tee_no_gt(DATA_PARAMS['base_path'], PREP_STEPS)

TEE IMAGES wo. gt:  120


In [55]:
tee_data_no_gt.files[0].name

'final_test_J49BJRA4.h5_ES.tif'

In [53]:
predict(unet, tee_data_no_gt)

FileNotFoundError: [Errno 2] No such file or directory: './final_test_predictions/pred_Data/extracted_TEE/final_test/final_test_J49BJRA4.h5_ES.tif'

# Save model

In [ ]:
file_name = 'Gauss_bringth_scale15_model'

# Save the model
save_result(unet, MODEL_PATH, file_name, accuracy_tee, average_dice_tee, class_dice_tee)